In [1]:
from llama_index.llms.together import TogetherLLM
from llama_index.core import Settings

together_llm = TogetherLLM(model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", 
api_key="87cac61702f938de91651d60977c8a2978163b011f9ba6e991f0e954b9c2c2ce")

Settings.llm = together_llm
Settings.chunk_size = 1024
Settings.chunk_overlap = 100
Settings.context_window = 8100

/home/chandanha/Documents/My_Documents/Excel_data_reader/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.readers.file import PandasExcelReader
import importlib
from pathlib import Path

base_reader = PandasExcelReader()
base_docs = base_reader.load_data(Path("Employee_Sample_Data.xlsx"))

In [3]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate

chat_text_qa_msgs = [
# Defines the system role message for the chat interaction like guidelines and tone for the AI model.
ChatMessage(
    role=MessageRole.SYSTEM,
    content=("""
        You are a AI Chatbot assistance to answer the question for the excel data i have given.
        Your goal is to answer questions as accurately as possible based on the instructions and context provided.
        After answering to the query at the end tell 'Let me Know if You Have any Queries'
        If the context is Outside the Excel date Don't answer related to the question.
        Just say that, 'The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only'.
        If the context is Outside the Excel Don't try to make up an answer.
        For general questions like 'How are you?' or 'Who are you?', respond accordingly,"""
    ),
),
# Defines the system user message for the chat interaction like query and passing relevent chunks to the AI model.
ChatMessage(
    role=MessageRole.USER,
    content=(
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
    ),
),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [5]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(base_docs)
VectorStoreIndex_query_engine = index.as_query_engine(text_qa_template=text_qa_template)

In [6]:
query_str = "How many different country employees are there in the excel"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, the different countries where employees are located are:\n\n1. United States\n2. China\n3. Brazil\n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='f9322972-4e65-45dc-8e06-64abdc00b67c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='5399851e-ed6e-48ea-a2e4-22d4f8f55b03', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='cc2cb3c10810872b88dedfb110652d7cf0086afa7c2d97108751cec0541dd1da'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='7f81934f-e1c5-47ea-86c3-b1c392e5c389', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='36e4491f351f8cb7671d92dd91

In [7]:
query_str = "Frame some Questions related to the excel data that i have given"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Here are some questions related to the Excel data:\n\n**Demographics**\n\n1. What is the average age of the employees?\n2. What is the most common ethnicity among the employees?\n3. What is the gender distribution of the employees?\n4. Which city has the most employees?\n\n**Job Information**\n\n1. What are the different job titles mentioned in the data?\n2. Which department has the most employees?\n3. What is the average salary for each job title?\n4. Which employee has been with the company the longest?\n\n**Location**\n\n1. Which country has the most employees?\n2. Which city has the highest average salary?\n3. What is the distribution of employees across different cities?\n\n**Salary and Bonus**\n\n1. What is the average salary for all employees?\n2. Which employee has the highest salary?\n3. What is the average bonus percentage for all employees?\n4. Which department has the highest average bonus percentage?\n\n**Tenure**\n\n1. What is the average tenure of all 

In [8]:
query_str = "What was the maximum Salary"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, the maximum salary is:\n\n* $249,801 (belonging to Natalia Santos, Vice President Human Resources Speciality Products)', source_nodes=[NodeWithScore(node=TextNode(id_='f63bc2b8-ac88-4640-808a-937ae02147dc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='876fd225-2bb8-465d-a5f0-c957169cf6d8', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='4f80ccd07bd4b50fb70e66936279e403b92e68954323451f79823e85a114429d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='dd4efe23-ccac-4309-8ed1-9ced42486a93', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='17f0cbbc0117757ab1bef34afdbc7a2ee8521c9de6b32

In [9]:
query_str = "What was the minimum Salary"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, the minimum salary is:\n\n* 41336 (held by Luke Martin, Analyst Finance Manufacturing)\n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='f9322972-4e65-45dc-8e06-64abdc00b67c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='5399851e-ed6e-48ea-a2e4-22d4f8f55b03', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='cc2cb3c10810872b88dedfb110652d7cf0086afa7c2d97108751cec0541dd1da'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='7f81934f-e1c5-47ea-86c3-b1c392e5c389', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='36e4491f351f8cb7671d92dd91f2d719d5a02

In [10]:
query_str = "What was the minimum, minimum and average salary"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here are the minimum, maximum, and average salary:\n\n* Minimum salary: $40,545 (Peyton Walker, Analyst Marketing Research & Development)\n* Maximum salary: $249,270 (Isabella Xi, Vice President Marketing Research & Development)\n* Average salary: $114,419.19 \n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 

In [11]:
query_str = "What was the minimum, minimum and average age"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here are the minimum, maximum, and average ages:\n\n* Minimum age: 25 (Luke Martin, E04332)\n* Maximum age: 65 (Bella Powell, E00163)\n* Average age: 45.5 \n\nLet me know if you have any further queries.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United St

In [13]:
query_str = "Explain in detail about the employee Hannah Hoang"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Dinh Technical Architect IT Manufacturing Male Asian 59 1997-11-29 99975 0.0 C

In [14]:
query_str = "What are all the different Job Title are there in the excel"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here are the different Job Titles:\n\n1. Manager Accounting\n2. Manager Finance\n3. Vice President Finance\n4. Manager Human Resources\n5. IT Coordinator\n6. Development Engineer\n7. Director Marketing\n8. Enterprise Architect\n9. Sr. Business Partner\n10. Engineering Manager\n11. Sr. Analyst\n12. Solutions Architect\n13. Computer Systems Manager\n14. Operations Engineer\n15. Director Sales\n16. Vice President Human Resources\n17. Systems Analyst\n18. Director Engineering\n19. Sr. Manger Finance\n20. Sr. Manger Human Resources\n21. Account Representative\n22. Network Administrator\n23. Service Desk Analyst\n24. Sr. Account Representative\n25. Analyst II\n26. Cloud Infrastructure Architect\n27. Controls Engineer\n28. Director Human Resources\n29. Sr. Analyst Sales\n30. Manager Sales\n31. Engineering Manager\n32. Sr. Manger IT\n33. Sr. Manger Marketing\n34. Sr. Manger Sales\n\nNote: Some job titles may have slight variations (e.g., "Sr

In [15]:
query_str = "What are all the different Departments are there in the excel"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided context information, the different departments mentioned in the Excel data are:\n\n1. Accounting\n2. Engineering\n3. Human Resources\n4. IT\n5. Finance\n6. Marketing\n7. Manufacturing\n8. Research & Development\n9. Sales\n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='dd4efe23-ccac-4309-8ed1-9ced42486a93', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='f63bc2b8-ac88-4640-808a-937ae02147dc', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='183a824dd2278c962dc3b9d69a4e55072d828afb13578215e0c10c8bc0eadbb2'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='0d8317c

In [17]:
query_str = "What are all the different Ethnicity are there in the excel"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, the different Ethnicities mentioned are:\n\n1. Asian\n2. Caucasian\n3. Latino\n4. Black\n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='f63bc2b8-ac88-4640-808a-937ae02147dc', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='876fd225-2bb8-465d-a5f0-c957169cf6d8', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='4f80ccd07bd4b50fb70e66936279e403b92e68954323451f79823e85a114429d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='dd4efe23-ccac-4309-8ed1-9ced42486a93', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='17f0cbbc0117757ab1bef34afdbc7a2ee852

In [19]:
query_str = "What is the average salary for each job title?"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here is the average salary for each job title:\n\n1. **Account Representative**: \n    * Average salary: $62411 (based on 3 employees: Madeline Watson, Caleb Xiong, and Joshua Gupta)\n2. **Analyst**: \n    * Average salary: $64451 (based on 7 employees: Peyton Walker, Parker Allen, Nora Nunez, Peyton Vasquez, Charlotte Baker, Jeremiah Hernandez, and Henry Green)\n3. **Analyst II**: \n    * Average salary: $59758 (based on 2 employees: Nora Nunez and Charlotte Baker)\n4. **Cloud Infrastructure Architect**: \n    * Average salary: $69332 (based on 1 employee: Nova Lin)\n5. **Computer Systems Manager**: \n    * Average salary: $84913 (based on 1 employee: Penelope Jordan)\n6. **Controls Engineer**: \n    * Average salary: $114775 (based on 2 employees: Ivy Desai and Camila Rogers)\n7. **Director**: \n    * Average salary: $173511 (based on 9 employees: Emily Davis, Luna Sanders, Bella Powell, Adam Dang, Leilani Thao, Elena Mendoza, Jose

In [20]:
query_str = "Which city has the highest number of employees in the United States?"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here is the answer to the question:\n\nThe city with the highest number of employees in the United States is:\n\n* Chicago (with 11 employees)\n\nHere is the breakdown of the number of employees in each city in the United States:\n\n* Chicago: 11\n* Seattle: 5\n* Phoenix: 3\n* Austin: 3\n* Columbus: 3\n* Miami: 2\n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15

In [18]:
"""
Response(response='Here are some potential questions related to the Excel data:\n\n**Demographics**\n\n
1. What is the average age of the employees in the dataset?\n
2. What is the most common ethnicity among the employees?\n
3. What is the distribution of male and female employees in the dataset?\n
4. Which city has the highest number of employees?\n\n**Job Information**\n\n
1. What are the most common job titles in the dataset?\n
2. Which department has the highest number of employees?\n
3. What is the average salary for each job title?\n
4. Which employee has the highest salary?\n\n**Location**\n\n
1. Which country has the highest number of employees?\n
2. Which city has the highest number of employees in the United States?\n
3. What is the distribution of employees across different regions (e.g. Asia, North America, etc.)?\n\n**Employment History**\n\n
1. What is the average length of employment for employees in the dataset?\n2
. Which employee has been with the company the longest?\n
3. What is the distribution of employees by hire date (e.g. by year, by month)?\n\n**Other**\n\n
1. What is the average bonus percentage for employees in the dataset?\n2. Which employee has the highest bonus percentage?\n
3. What is the distribution of employees by job level (e.g. manager, director, etc.)?\n\nLet me know if you have any queries!'
"""

"\nResponse(response='Here are some potential questions related to the Excel data:\n\n**Demographics**\n\n\n1. What is the average age of the employees in the dataset?\n\n2. What is the most common ethnicity among the employees?\n\n3. What is the distribution of male and female employees in the dataset?\n\n4. Which city has the highest number of employees?\n\n**Job Information**\n\n\n1. What are the most common job titles in the dataset?\n\n2. Which department has the highest number of employees?\n\n3. What is the average salary for each job title?\n\n4. Which employee has the highest salary?\n\n**Location**\n\n\n1. Which country has the highest number of employees?\n\n2. Which city has the highest number of employees in the United States?\n\n3. What is the distribution of employees across different regions (e.g. Asia, North America, etc.)?\n\n**Employment History**\n\n\n1. What is the average length of employment for employees in the dataset?\n2\n. Which employee has been with the com

In [21]:
query_str = "Give a complete detail about the employee who has the highest Salary?"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here is the information about the employee with the highest salary:\n\n**Employee Details:**\n\n1. **Employee ID:** E00549\n2. **Name:** Isabella Xi\n3. **Job Title:** Vice President Marketing\n4. **Department:** Research & Development\n5. **Gender:** Female\n6. **Ethnicity:** Asian\n7. **Age:** 41\n8. **Hire Date:** 2013-03-13\n9. **Salary:** 249270\n10. **Bonus:** 0.3\n11. **Location:** United States, Seattle\n12. **Termination Date:** NaT (Not Applicable)\n\nLet me know if you have any further queries.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.N

In [22]:
query_str = "Which city has the highest number of employees in the United States?"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='Based on the provided Excel data, here is the answer to the question:\n\nThe city with the highest number of employees in the United States is:\n\n* Chicago (with 11 employees)\n\nHere is the breakdown of the number of employees in each city in the United States:\n\n* Chicago: 11\n* Seattle: 4\n* Phoenix: 3\n* Austin: 3\n* Miami: 2\n* Columbus: 2\n* Other cities: 1 or less\n\nLet me know if you have any queries.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1

In [23]:
query_str = "Explain about cricket"
response = VectorStoreIndex_query_engine.query(query_str)
response

Response(response='The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Dinh Technical Architect IT Manufacturing Male Asian 59 1997-11-29 99975 0.0 C

In [24]:
query_str = "Give a complete detail about the employee with employee ID E04598"
response = VectorStoreIndex_query_engine.query(query_str)
response 

Response(response='The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only.\n\nThere is no employee with employee ID E04598 in the provided Excel data.', source_nodes=[NodeWithScore(node=TextNode(id_='7dd17d9a-a464-4d52-a358-b995331989f6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c8f8d709-f654-465a-99e8-a46c96fe4c5f', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Di

In [25]:
from llama_index.core import SummaryIndex

base_index = SummaryIndex.from_documents([base_docs[0]])
SummaryIndex_base_query_engine = base_index.as_query_engine(text_qa_template=text_qa_template)

In [26]:
query_str = "How many different country employees are there in the excel"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='There are 3 different countries where employees are located:\n\n1. United States\n2. China\n3. Brazil', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Dinh Technical Architect IT Manufacturing Male Asian 59 1997-11-29 99975 0.0 China Chongqing Na

In [27]:
query_str = "What was the maximum Salary"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='The absolute maximum salary is 258498', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Dinh Technical Architect IT Manufacturing Male Asian 59 1997-11-29 99975 0.0 China Chongqing NaT\nE02572 Luna Sanders Director Finance Speciality Products Fema

In [28]:
query_str = "Explain about cricket"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Cricket is a popular team sport played with a bat and ball on a rectangular field with two sets of three stumps (called wickets) at each end. It is one of the most widely played sports in the world, with a huge following in countries like India, Australia, Pakistan, England, and South Africa.\n\nThe objective of the game is to score runs by hitting the ball with the bat and running between the wickets, while the opposing team tries to stop them by getting the batsmen out. A match is typically played between two teams, each with 11 players.\n\nThere are several formats of cricket, including Test matches, One-Day Internationals (ODIs), and Twenty20 (T20) matches. Test matches are the longest format, lasting up to five days, while T20 matches are the shortest, lasting about three hours.\n\nCricket has a rich history, dating back to the 16th century in England. It has evolved over the years, with changes in rules, equipment, and playing styles. Today, cricket is a global

In [ ]:
query_str = "What was the maximum Salary"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

### Questions

- What was the minimum, minimum and average salary
- What was the minimum, minimum and average age
- Explain in detail about the employee Hannah Hoang
- What are all the different Job Title are there in the excel
- What are all the different Departments are there in the excel
- What are all the different Ethnicity are there in the excel
- Give a complete detail about the employee who is taking maximum salary
- Give a complete detail about the employee who is taking minimum salary
- What is the average salary for each job title?
- Which city has the highest number of employees in the United States?

In [29]:
query_str = "What was the minimum, minimum and average salary"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Based on the provided context information, here are the minimum, maximum, and average salaries:\n\n* Minimum salary: 38,609 \n* Maximum salary: 258,498 \n* Average salary: 103,819.19', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Dinh Technical

In [30]:
query_str = "What was the minimum, minimum and average age"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Based on the provided context information, here are the minimum, maximum, and average ages:\n\n* Minimum age: 25 (Multiple employees, including Maria Sun, Director Sales Corporate; Hannah Hoang, Manager Accounting Speciality Products)\n* Maximum age: 63 (Sofia Cheng, Vice President Accounting Corporate)\n* Average age: 44.6 (calculated based on the ages of all employees)', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f3

In [31]:
query_str = "What was the minimum, minimum and average age"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Based on the provided context information, here are the minimum, maximum, and average ages:\n\n* Minimum age: 25 (E02191 Maria Sun, E00785 Hannah Hoang, E02558 Jose Richardson, E02661 Maya Chan, E00956 Eleanor Chau, E03278 Nicholas Wong)\n* Maximum age: 63 (E03545 Sofia Cheng)\n* Average age: 44.6 (calculated by summing up all the ages and dividing by the total number of employees)', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e079

In [32]:
query_str = "Explain in detail about the employee Hannah Hoang"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Hannah Hoang is a Manager in the Accounting department of Speciality Products, based in China, Chengdu. She is 25 years old and has been with the company since December 15, 2021. Her employee ID is E00785.', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 

In [33]:
query_str = "What are all the different Job Title are there in the excel"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Here are the different job titles mentioned in the data:\n\n1. Manager\n2. Director\n3. Sr. Manager\n4. Sr. Analyst\n5. Analyst\n6. Analyst II\n7. Vice President\n8. Solutions Architect\n9. IT Systems Architect\n10. Network Architect\n11. Technical Architect\n12. Systems Analyst\n13. Network Administrator\n14. Field Engineer\n15. Controls Engineer\n16. Development Engineer\n17. Business Partner\n18. Sr. Business Partner\n19. Enterprise Architect\n20. Quality Engineer\n21. Operations Engineer\n22. Automation Engineer\n23. System Administrator\n24. Account Representative\n25. Sr. Account Representative\n26. Cloud Infrastructure Architect\n27. Computer Systems Manager\n28. HRIS Analyst\n29. Service Desk Analyst\n30. Engineering Manager', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNod

In [34]:
query_str = "What are all the different Ethnicity are there in the excel"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Based on the provided context information, the different Ethnicities mentioned in the Excel data are:\n\n1. Asian\n2. Black\n3. Caucasian\n4. Latino', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7718e7f39f0a57fdf38d005490d91')}, text='E02387 Emily Davis Sr. Manger IT Research & Development Female Black 55 2016-04-08 141604 0.15 United States Seattle 2021-10-16\nE04105 Theodore Dinh Technical Architect IT Manufacturing Male A

In [35]:
query_str = "Give a complete detail about the employee who is taking maximum salary"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Based on the provided context information, the employee who is taking the maximum salary is:\n\n**Raelynn Rios**\n\nHere are the details about Raelynn Rios:\n\n1. Employee ID: E04354\n2. Job Title: Vice President Sales Manufacturing\n3. Department: Sales\n4. Location: United States Columbus\n5. Gender: Female\n6. Ethnicity: Latino\n7. Age: 43\n8. Hire Date: 2016-08-21\n9. Salary: 258498\n10. Bonus: 0.35', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184

In [36]:
query_str = "Give a complete detail about the employee who is taking minimum salary"
base_response = SummaryIndex_base_query_engine.query(query_str)
base_response

Response(response='Based on the provided context information, here are the details about the employee who is taking the minimum salary:\n\n1. Employee ID: E01148\n2. Name: Scarlett Kumar\n3. Job Title: Systems Analyst\n4. Department: IT Corporate\n5. Location: United States Columbus\n6. Salary: 47032\n7. Bonus: 0.0\n8. Age: 55\n9. Gender: Female\n10. Ethnicity: Asian\n11. Date of Hire: 2009-01-07', source_nodes=[NodeWithScore(node=TextNode(id_='f467afbc-5639-4822-8d2e-aa0b3d5218b4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='816be33c-3160-4c5d-8d9d-85ba970f190d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='96f35758b9a19493171d6f94910135dec868b8d9c742b4bb2e63bfa7d1fa544d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='6d2ba9e7-41a4-49a8-8c46-d870c60fce40', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='1ce7e63184b573bdacf7d113be15e07904f7